In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
dataset = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
dataset.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


In [3]:
import cv2

In [4]:
PATH = "../input/plant-pathology-2021-fgvc8/train_images/"

In [5]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(PATH+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [6]:
load_image('800cbf0ff87721f8.jpg').shape

(2672, 4000, 3)

In [7]:
def cut_leaf(img):
    emb_img = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2BGR)
    edges = np.asarray(cv2.Canny(img, 100, 200))
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))
    
    edge_coors = np.asarray(edge_coors)
    
    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img[row_min:row_max, col_min:col_max]
    
    return new_img

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
lbEncode = LabelEncoder()

In [10]:
dataset['labels'] = lbEncode.fit_transform(dataset['labels'])

In [11]:
dataset.iloc[:, 1:].value_counts()

labels
9         4826
3         4624
1         3181
6         1860
0         1602
4         1184
10         686
11         200
2          165
8          120
7           97
5           87
dtype: int64

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

In [13]:
augmentations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor()
])

In [14]:
def preprocess(img):
    image = load_image(img)
    image = cv2.resize(image, (224, 224))
    image = torch.tensor(image, dtype=torch.float32) / 255
    image = image.reshape(3, 224, 224)
    
    return image

In [15]:
class Data(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.image_id = X
        self.labels = y
        self.transforms = transform
    
    def __len__(self):
        return len(self.image_id)
    
    def __getitem__(self, idx):
        if self.labels is not None:
            image_id = self.image_id[idx][0]
            label = self.labels[idx]
        
            image = preprocess(image_id)
            if self.transforms:
                image = self.transforms(image)
        
            return image, label
        else:
            image_id = self.image_id[idx][0]
        
            image = preprocess(image_id)
            if self.transforms:
                image = self.transforms(image)
            
            return image

In [16]:
train_dataset = Data(dataset.iloc[:, :1].values,dataset.iloc[:, 1:].values, transform=augmentations)
train_loader = DataLoader(train_dataset, batch_size=16)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
epochs = 5
learning_rate = 0.001

In [19]:
class bottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1, dim_change=None):
        super(bottleNeck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, stride=1, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, stride=stride, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes*self.expansion, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(planes*self.expansion)
        self.dim_change = dim_change
    
    def forward(self, x):
        res = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        
        if self.dim_change is not None:
            res = self.dim_change(res)
            
        out += res
        out = F.relu(out)
        
        return out

In [20]:
class baseBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, dim_change=None):
        super(baseBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, stride=stride, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, stride=1, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dim_change = dim_change
    def forward(self, x):
        res = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.dim_change is not None:
            res = self.dim_change(res)
        
        out += res
        out = F.relu(out)
        
        return out

In [21]:
class ResNet(nn.Module):
    def __init__(self, block, num_layers, num_classes=12):
        super(ResNet, self).__init__()
        
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._layer(block, 64, num_layers[0], stride=1)
        self.layer2 = self._layer(block, 128, num_layers[1], stride=2)
        self.layer3 = self._layer(block, 256, num_layers[2], stride=2)
        self.layer4 = self._layer(block, 512, num_layers[3], stride=2)
        self.avgPool = nn.AvgPool2d(kernel_size=4, stride=1)
        self.fc = nn.Linear(25088, num_classes)
        
    def _layer(self, block, planes, num_layers, stride=1):
        dim_change = None
        if stride != 1 or planes != self.in_planes * block.expansion:
            dim_change = nn.Sequential(nn.Conv2d(self.in_planes, planes*block.expansion, kernel_size=1, stride=stride),
                                       nn.BatchNorm2d(planes*block.expansion))
        netLayers = []
        netLayers.append(block(self.in_planes, planes, stride=stride, dim_change=dim_change))
        self.in_planes = planes * block.expansion
        for i in range(1, num_layers):
            netLayers.append(block(self.in_planes, planes))
            self.in_planes = planes * block.expansion
            
        return nn.Sequential(*netLayers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = F.avg_pool2d(x, 4)
        x = x.flatten(1)
        x = self.fc(x)
        
        return x

In [22]:
model = ResNet(baseBlock, [3, 4, 6, 3]).to(device)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
n_total_steps = len(train_loader)

In [25]:
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.flatten()
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch: [{epoch+1}/{epochs}], Step: [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch: [1/5], Step: [10/1165], Loss: 5.1124
Epoch: [1/5], Step: [20/1165], Loss: 5.3211
Epoch: [1/5], Step: [30/1165], Loss: 1.8668
Epoch: [1/5], Step: [40/1165], Loss: 3.4333
Epoch: [1/5], Step: [50/1165], Loss: 8.3857
Epoch: [1/5], Step: [60/1165], Loss: 4.5563
Epoch: [1/5], Step: [70/1165], Loss: 2.8042
Epoch: [1/5], Step: [80/1165], Loss: 1.9221
Epoch: [1/5], Step: [90/1165], Loss: 2.2487
Epoch: [1/5], Step: [100/1165], Loss: 2.1675
Epoch: [1/5], Step: [110/1165], Loss: 2.1451
Epoch: [1/5], Step: [120/1165], Loss: 2.0109
Epoch: [1/5], Step: [130/1165], Loss: 2.2206
Epoch: [1/5], Step: [140/1165], Loss: 1.9003
Epoch: [1/5], Step: [150/1165], Loss: 1.8641
Epoch: [1/5], Step: [160/1165], Loss: 1.7733
Epoch: [1/5], Step: [170/1165], Loss: 1.8482
Epoch: [1/5], Step: [180/1165], Loss: 2.0079
Epoch: [1/5], Step: [190/1165], Loss: 2.9402
Epoch: [1/5], Step: [200/1165], Loss: 1.8190
Epoch: [1/5], Step: [210/1165], Loss: 1.4691
Epoch: [1/5], Step: [220/1165], Loss: 1.6970
Epoch: [1/5], Step:

In [26]:
torch.save(model.state_dict(), 'model.pth')